In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [8]:
df=pd.read_csv("Churn_Modelling.csv")  #Employe will leave or not job
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [11]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [12]:
#Drop unimportant column

df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [13]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [14]:
df['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [15]:
x=pd.get_dummies(df.drop('Exited',axis=1),drop_first=True)
y=df['Exited']

In [16]:
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [17]:
## spilitting the dataset into the Training set and test set

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)


In [18]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((8000, 11), (2000, 11), (8000,), (2000,))

In [19]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

# Modelling

# Initializing the Artificial Neural Network

In [20]:
from keras.models import Sequential
ann=Sequential()


# Adding the input layer and the first hidden layer

In [21]:
from keras.layers import Dense
ann.add(Dense(input_dim=11,units=6,kernel_initializer='uniform',activation='relu'))
#ann.add(Dropoout(rate=0.1))


# Adding the Second hidden layer

In [22]:
ann.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))

#ann.add(Dropout(rate=0.1))

# Adding Output layer

In [23]:

ann.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

# Compilling the ANN

In [24]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Training the ANN on the Training set

In [25]:
ann.fit(x_train,y_train,epochs=100,batch_size=32)

Epoch 1/100


250/250 [==============================] - 4s 4ms/step - loss: 0.5770 - accuracy: 0.7947
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4362 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4311 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4285 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4261 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4234 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4210 - accuracy: 0.7986
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4189 - accuracy: 0.8195
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4175 - accuracy: 0.8236
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4161 - accu

Epoch 77/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4000 - accuracy: 0.8356
Epoch 78/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4002 - accuracy: 0.8344
Epoch 79/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4002 - accuracy: 0.8347
Epoch 80/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4002 - accuracy: 0.8364
Epoch 81/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3999 - accuracy: 0.8359
Epoch 82/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4000 - accuracy: 0.8350
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3999 - accuracy: 0.8365
Epoch 84/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3997 - accuracy: 0.8364
Epoch 85/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4000 - accuracy: 0.8354
Epoch 86/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3998

# Predictions & Evaluating the model

# Prediction

In [28]:
ypred_train=ann.predict(x_train)
ypred_test=ann.predict(x_test)
ypred_train=(ypred_train>0.5)
ypred_test=(ypred_test>0.5)


63/63 [==============================] - 0s 3ms/step


# Evaluation

In [29]:
from sklearn.metrics import confusion_matrix,accuracy_score

print("Test Accuracy",accuracy_score(y_test,ypred_test))
print("train Accuracy",accuracy_score(y_train,ypred_train))

confusion_matrix(y_test,ypred_test)
      

Test Accuracy 0.8425
train Accuracy 0.835625


array([[1543,   52],
       [ 263,  142]], dtype=int64)

# Cross validate the model

In [30]:
def build_cross_classifier():
    classifier=Sequential()
    classifier.add(Dense(input_dim=11,units=6,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))
    classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return classifier

In [31]:
#pip install scikeras

In [32]:
from scikeras.wrappers import KerasClassifier
classifier = KerasClassifier(build_cross_classifier,batch_size=32,epochs=100)

In [33]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(classifier,x,y,cv=5)

Epoch 1/100
250/250 [==============================] - 3s 4ms/step - loss: 0.5638 - accuracy: 0.7894
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5265 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5187 - accuracy: 0.7964
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5117 - accuracy: 0.7964
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5067 - accuracy: 0.7964
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5071 - accuracy: 0.7964
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5065 - accuracy: 0.7964
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5058 - accuracy: 0.7964
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5039 - accuracy: 0.7964
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5040 - accura

250/250 [==============================] - 1s 4ms/step - loss: 0.4971 - accuracy: 0.7964
Epoch 82/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4974 - accuracy: 0.7964
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4984 - accuracy: 0.7964
Epoch 84/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5047 - accuracy: 0.7964
Epoch 85/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5041 - accuracy: 0.7964
Epoch 86/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5037 - accuracy: 0.7964
Epoch 87/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5037 - accuracy: 0.7964
Epoch 88/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5039 - accuracy: 0.7964
Epoch 89/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5041 - accuracy: 0.7964
Epoch 90/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5040 - accuracy: 

250/250 [==============================] - 1s 4ms/step - loss: 0.4989 - accuracy: 0.7964
Epoch 62/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4983 - accuracy: 0.7964
Epoch 63/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4976 - accuracy: 0.7964
Epoch 64/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4976 - accuracy: 0.7964
Epoch 65/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4973 - accuracy: 0.7964
Epoch 66/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4972 - accuracy: 0.7964
Epoch 67/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4967 - accuracy: 0.7964
Epoch 68/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4973 - accuracy: 0.7964
Epoch 69/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4971 - accuracy: 0.7964
Epoch 70/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4974 - accuracy: 

250/250 [==============================] - 1s 4ms/step - loss: 0.4991 - accuracy: 0.7962
Epoch 42/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4998 - accuracy: 0.7962
Epoch 43/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4985 - accuracy: 0.7962
Epoch 44/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4987 - accuracy: 0.7962
Epoch 45/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4992 - accuracy: 0.7962
Epoch 46/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4989 - accuracy: 0.7962
Epoch 47/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4989 - accuracy: 0.7962
Epoch 48/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4984 - accuracy: 0.7962
Epoch 49/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4998 - accuracy: 0.7962
Epoch 50/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4995 - accuracy: 

250/250 [==============================] - 1s 4ms/step - loss: 0.5010 - accuracy: 0.7962
Epoch 22/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5003 - accuracy: 0.7962
Epoch 23/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5025 - accuracy: 0.7962
Epoch 24/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5023 - accuracy: 0.7962
Epoch 25/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5008 - accuracy: 0.7962
Epoch 26/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5008 - accuracy: 0.7962
Epoch 27/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5000 - accuracy: 0.7962
Epoch 28/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5006 - accuracy: 0.7962
Epoch 29/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4999 - accuracy: 0.7962
Epoch 30/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5007 - accuracy: 

63/63 [==============================] - 0s 3ms/step
Epoch 1/100
250/250 [==============================] - 3s 4ms/step - loss: 0.5375 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5251 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5187 - accuracy: 0.7962
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5112 - accuracy: 0.7962
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5093 - accuracy: 0.7962
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5086 - accuracy: 0.7962
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5092 - accuracy: 0.7962
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5085 - accuracy: 0.7962
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5077 - accuracy: 0.7962
Epoch 10/100
250/250 [================

250/250 [==============================] - 1s 4ms/step - loss: 0.5047 - accuracy: 0.7962
Epoch 81/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5042 - accuracy: 0.7962
Epoch 82/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5014 - accuracy: 0.7962
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4989 - accuracy: 0.7962
Epoch 84/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5047 - accuracy: 0.7962
Epoch 85/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5048 - accuracy: 0.7962
Epoch 86/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5045 - accuracy: 0.7962
Epoch 87/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5048 - accuracy: 0.7962
Epoch 88/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5048 - accuracy: 0.7962
Epoch 89/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5047 - accuracy: 

In [34]:
print(accuracies)
accuracies.mean()

[0.796  0.796  0.7965 0.7965 0.7965]


0.7963

# Improving and tunning the ANN

# This can be done using three options

* Hyperparameter Tunning
* Regularization(L1&L2) to reduce overfitting if needed(for Regression problem)
* Dropout(mostly use)

# Hyperparameter tuning can be done for identifying no.of hidden layers & no.of neurons in seach hidden layer

* layers=[[20],[10],[30],[40]]--->1 hidden layer with different options of no.of neurons in that hidden layer
* layers=[[20],[40,20],[45,30],[15]]-->Multiple hidden layers with different options of no.of neurons

# Hyperparameter tunning can be done for identifying best activation fuction for hidden layers

* activations=['relu','sigmoid']

# Hyperparameter tunning can be done for identifie best optimizers
* optimizer =['adam','rmsprop']

# Hyperparameter tuning can be done for identifying best batchsize for building/training model

* batchsize=[10,16,32,64,128,256]

In [35]:
estimator=KerasClassifier(build_cross_classifier())
param_grid={'batch_size':[10,32],'epochs':[50,100],"optimizer":['adam','rmsprop']}


In [ ]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(estimator,param_grid,scoring='accuracy',cv=5)
grid_result=grid.fit(x_train,y_train)

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp31gtv432\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp31gtv432\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp_hkov1z4\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp_hkov1z4\assets


Epoch 1/50
640/640 [==============================] - 4s 4ms/step - loss: 0.5004 - accuracy: 0.7950
Epoch 2/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4314 - accuracy: 0.7959
Epoch 3/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4269 - accuracy: 0.7959
Epoch 4/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4202 - accuracy: 0.8158
Epoch 5/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4153 - accuracy: 0.8298
Epoch 6/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4116 - accuracy: 0.8327
Epoch 7/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4084 - accuracy: 0.8328
Epoch 8/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4067 - accuracy: 0.8345
Epoch 9/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4048 - accuracy: 0.8347
Epoch 10/50
640/640 [==============================] - 3s 4ms/step - loss: 0.4037 - accuracy: 0.8348

...

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmp85ta3f6g\assets


Epoch 1/100
640/640 [==============================] - 4s 4ms/step - loss: 0.5016 - accuracy: 0.7953
Epoch 2/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4317 - accuracy: 0.7961
Epoch 3/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4274 - accuracy: 0.7961
Epoch 4/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4227 - accuracy: 0.7961
Epoch 5/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4199 - accuracy: 0.8089
Epoch 6/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4170 - accuracy: 0.8234
Epoch 7/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4157 - accuracy: 0.8291
Epoch 8/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4139 - accuracy: 0.8305
Epoch 9/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4128 - accuracy: 0.8328
Epoch 10/100
640/640 [==============================] - 3s 4ms/step - loss: 0.4120 - accura

640/640 [==============================] - 3s 4ms/step - loss: 0.3989 - accuracy: 0.8372
Epoch 82/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3991 - accuracy: 0.8366
Epoch 83/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3990 - accuracy: 0.8366
Epoch 84/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3991 - accuracy: 0.8367
Epoch 85/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3990 - accuracy: 0.8361
Epoch 86/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3991 - accuracy: 0.8361
Epoch 87/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3988 - accuracy: 0.8377
Epoch 88/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3990 - accuracy: 0.8364
Epoch 89/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3989 - accuracy: 0.8355
Epoch 90/100
640/640 [==============================] - 3s 4ms/step - loss: 0.3992 - accuracy: 

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpp0bsx62a\assets


Epoch 1/100
640/640 [==============================] - 1s 2ms/step - loss: 0.4983 - accuracy: 0.7956
Epoch 2/100
640/640 [==============================] - 2052s 3s/step - loss: 0.4285 - accuracy: 0.7961
Epoch 3/100
640/640 [==============================] - 4s 7ms/step - loss: 0.4232 - accuracy: 0.7961
Epoch 4/100
640/640 [==============================] - 5s 9ms/step - loss: 0.4197 - accuracy: 0.7961
Epoch 5/100
640/640 [==============================] - 2s 4ms/step - loss: 0.4165 - accuracy: 0.8263
Epoch 6/100
640/640 [==============================] - 2s 3ms/step - loss: 0.4152 - accuracy: 0.8248
Epoch 7/100
640/640 [==============================] - 2s 3ms/step - loss: 0.4135 - accuracy: 0.8327
Epoch 8/100
640/640 [==============================] - 1s 2ms/step - loss: 0.4124 - accuracy: 0.8316
Epoch 9/100
640/640 [==============================] - 1s 2ms/step - loss: 0.4110 - accuracy: 0.8330
Epoch 10/100
640/640 [==============================] - 2s 3ms/step - loss: 0.4100 - accu

In [1]:
#best parameters
grid_result.best_params_

NameError: name 'grid_result' is not defined

In [2]:
grid_result.best_score_

NameError: name 'grid_result' is not defined